In [21]:
ls

SKU_to_product_support_doc.json       index.html
Verifying Product Support Docs.ipynb


In [22]:
file_path = 'SKU_to_product_support_doc.json'

In [18]:
#@title Connect/Log-In to Odoo

import xmlrpc.client

# Your Odoo connection details
# We either need the password or the api key
url = 'https://mid-city-engineering.odoo.com/'       # Replace with your Odoo URL
db = 'mid-city-engineering'                          # Your database name
username = 'sales@midcityengineering.com'            # Your login email
api_key = '3eb5c73cc01b4d5279d20a6f727b53dfab0a01f6' # Your API key
password = api_key                                   # Your login password

common = xmlrpc.client.ServerProxy(f'{url}/xmlrpc/2/common')
# This API call takes the url but no credential data, so it is public

 # To verify if the connection information is correct before trying to
 # authenticate, the simplest call is to ask for the server’s version.
print('Values returned by common.version():')
print(common.version())
print('\n-----------\n\nDescription of the returned common object:')
type(common)

# Once the url is validated, we use the common.authenticate function to log in

uid = common.authenticate(db, username, api_key, {}) # use api_key or password
print(f"Connected successfully! User ID: {uid}")
models = xmlrpc.client.ServerProxy(f'{url}/xmlrpc/2/object')

Values returned by common.version():
{'server_version': 'saas~18.2+e', 'server_version_info': ['saas~18', 2, 0, 'final', 0, 'e'], 'server_serie': 'saas~18.2', 'protocol_version': 1}

-----------

Description of the returned common object:
Connected successfully! User ID: 10


In [23]:
import json

# Load your data
with open(file_path, 'r') as f:
    data = json.load(f)

# Total SKUs
total_skus = len(data)

# Unique SKUs
unique_skus = len(set(item['SKU'] for item in data))

print(f"Total SKUs: {total_skus}")
print(f"Unique SKUs: {unique_skus}")

# If there are duplicates, you might want to see them:
if total_skus != unique_skus:
    from collections import Counter
    sku_counts = Counter(item['SKU'] for item in data)
    duplicates = {sku: count for sku, count in sku_counts.items() if count > 1}
    print(f"\nDuplicate SKUs found: {len(duplicates)}")
    for sku, count in duplicates.items():
        print(f"  {sku}: appears {count} times")

Total SKUs: 75
Unique SKUs: 73

Duplicate SKUs found: 2
  ECOV2: appears 2 times
  ECOPCM: appears 2 times


In [24]:
# Extract SKU list from your JSON
sku_list = [item['SKU'] for item in data]

# Search for published products with matching default_code
product_ids = models.execute_kw(
    db, uid, password,
    'product.template', 'search',
    [[
      ['is_published', '=', True],
      # ['default_code', 'in', sku_list]
    ]]
)

print(f"Found {len(product_ids)} published products")
print(f"Expected: {len(sku_list)} ({len(set(sku_list))} unique)")

# Get details of found products
if product_ids:
    products = models.execute_kw(
        db, uid, password,
        'product.template', 'read',
        [product_ids],
        {'fields': ['id', 'name', 'default_code', 'is_published']}
    )

    # Check which SKUs are missing
    found_skus = {p['default_code'] for p in products if p['default_code']}
    missing_skus = set(sku_list) - found_skus

    if missing_skus:
        print(f"\n⚠️ Missing {len(missing_skus)} SKUs:")
        for sku in sorted(missing_skus):
            print(f"  - {sku}")

Found 130 published products
Expected: 75 (73 unique)

⚠️ Missing 2 SKUs:
  - MAXINT (All SKUS)
  - MOTOPLATE (All SKUS)


In [25]:
MAXINT_products = models.execute_kw(
    db, uid, password,
    'product.template', 'search_read',
    [[
        ['is_published', '=', True],
        ['default_code', 'ilike', '%MAXINT%'],
    ]],
    {'fields': ['id', 'name', 'default_code']}
)

In [26]:
MAXINT_SKUs = [p['default_code'] for p in MAXINT_products if p['default_code']]
MAXINT_SKUs

['MAXINTHUR',
 'MAXINT125',
 'MAXINT213',
 'MAXINTF250',
 'MAXINT221',
 'MAXINT222',
 'MAXINTA29',
 'MAXINTA29V2',
 'MAXINTA8',
 'MAXINTFUS',
 'MAXINTJGC',
 'MAXINTJXE',
 'MAXINTMDX',
 'MAXINTP11',
 'MAXINTP29V2',
 'MAXINTP29V3',
 'MAXINTP29V4',
 'MAXINTP29V5',
 'MAXINTX1',
 'MAXINTP29']

In [27]:
MOTOPLATE_products = models.execute_kw(
    db, uid, password,
    'product.template', 'search_read',
    [[
        ['is_published', '=', True],
        ['default_code', 'ilike', '%MOTO%'],
    ]],
    {'fields': ['id', 'name', 'default_code']}
)

In [28]:
MOTOPLATE_SKUs = [p['default_code'] for p in MOTOPLATE_products if p['default_code']]
MOTOPLATE_SKUs

['MOTO212', 'MOTO213', 'MOTO222', 'MOTOTS3']